# Extraction du graphe pour les données de Hurtubise

## Récupération des données

In [36]:
class ISBN:
    def __init__(self,papier = "", pdf = "", epub = ""):
        self.papier = papier
        self.pdf = pdf
        self.epub = epub
    

class Livre:
    def __init__(self):
        self.editeur = ""
        self.isbn = None
        self.titre = ""
        self.sous_titre = ""
        self.titre_serie = ""
        self.contributeurs = ""
        self.auteur = []
        self.langue = ""
        self.langue_origine = ""
        self.resume = ""
        self.nombre_pages = ""
        self.annee_parution = ""
        self.thema_principal = ""
        self.thema_autre = ""
        self.quant_geo = ""
        self.quant_langue = ""
        self.quant_hist = ""
        self.niveau_scolaire = ""
        self.niveau_lecture = ""
        self.echelle_CECR = ""
        self.quant_interet = ""
        self.quant_age = ""
        self.quant_style = ""
        self.classification_editoriale = ""
        self.mots_cles = []
        
        

In [48]:
f = open('../Data/Hurtubise/Exportation-Hurtubise-Nettoye.txt')
lignes = f.readlines()

livres = []

for ligne in lignes:
    items = ligne.split('<>')
    livre = Livre()
    livre.editeur = items[0]
    
    # Il y a trois numéros ISBN, dans cet ordre:  papier, pdf et epub
    livre.isbn = ISBN(items[1],items[2],items[3])
    livre.titre = items[4]
    livre.sous_titre = items[5]
    livre.titre_serie = items[6]
    livre.contributeurs = items[7]
    livre.auteur = [nom.strip() for nom in items[8].split(', ')]
    livre.langue = items[9]
    livre.langue_origine = items[10]
    livre.resume = items[11]
    if len(items) > 12:
        livre.nombre_pages = items[12]
        # on ne prend pas la date de parution (info trop précise) qui se trouve à la position items[13]
        livre.annee_parution = items[14]
        livre.thema_principal = items[15]
        livre.thema_autre = items[16]
        livre.quant_geo = items[17]
        livre.quant_langue = items[18]
        livre.quant_hist = items[19]
        # on ne prend pas les niveaux et cycle scolaire de la France (en positions 20 et 22)
        livre.niveau_scolaire = items[21]
        livre.niveau_lecture = items[23]
        livre.echelle_CECR = items[24]
        livre.quant_interet = items[25]
        livre.quant_age = items[26]
        livre.quant_style = items[27]
        livre.classification_editoriale = items[28]
        livre.mots_cles = [mot.strip() for mot in items[29].strip().split(', ') if mot.strip() != '']
    livres.append(livre)
   

In [49]:
print(len(livres), 'livres au total')

1313 livres au total


In [50]:
# statistiques  sur les mots clés

mots_cles = {}

for l in livres:
    if l.mots_cles:
        for mot in l.mots_cles:
            if mot in mots_cles:
                mots_cles[mot] += 1
            else:
                mots_cles[mot] = 1
            
liste_mots_cles = sorted([(nombre, mot) for (mot, nombre) in mots_cles.items()], reverse = True)

# for mot in liste_mots_cles[:100]:
#      print(mot)


In [52]:
# Statistiques sur les Thema

themas = {}

for l in livres:
    if l.thema_principal:
        if l.thema_principal in themas:
            themas[l.thema_principal] += 1
        else:
            themas[l.thema_principal] = 1
            
liste_themas = sorted([(nombre, mot) for (mot, nombre) in themas.items()], reverse = True)

for thema in liste_themas[:100]:
     print(thema)


(117, 'FV - Roman historique')
(59, 'YFC - Fiction jeunesse : romans d’action et d’aventures')
(35, 'DNL - Essais littéraires')
(26, 'FBA - Littérature moderne et contemporaine')
(24, 'YFCF - Fiction jeunesse : romans policiers et à suspense')
(22, 'YBCS - Livres d’histoires illustrés')
(20, 'YFN - Fiction jeunesse : histoires familiales et de la vie de tous les jours')
(18, 'YFT - Fiction jeunesse : romans historiques')
(15, 'YFCB - Fiction jeunesse : thrillers')
(15, 'CJBG - Apprentissage des langues : grammaire, vocabulaire et prononciation')
(13, 'JNT - Compétences et techniques éducatives')
(13, 'FYB - Nouvelles')
(12, 'FS - Roman abordant la vie familiale')
(10, 'VFXB - Grossesse, naissance et soins du bébé : conseils et questions')
(9, 'YNT - Documentaires jeunesse : sciences et technologie')
(9, 'YFM - Fiction jeunesse : romans sentimentaux, d’amour et histoires de relation')
(9, 'YFH - Fiction jeunesse : fantasy')
(7, 'YFA - Fiction jeunesse : les classiques')
(7, 'DNBL - Biog

## Code pour l'extraction des contributeurs d'un livre

In [ ]:
# typesContributeurs = [
    '(De (auteur))', 
    '(Coordination éditoriale de)', 
    '(Photographies de)',
    '(Postface de)',
    '(Préface de)',
    '(De (photographe))',
    '(Traduit par)',
    '(Avant-propos de)',
    '(Illustré par)',
    '(Texte de)',
    '(Adapté par)']
 

def extraireContributeur(chaine):
    for c in typesContributeurs:
        position = chaine.strip().find(c)
        if position != -1:
            return chaine.strip()[:position-1]
           
    return ""

    
def extraireTypeContributeur(chaine):
    for c in typesContributeurs:
        position = chaine.strip().find(c)
        if position != -1:
            return chaine.strip()[position:]
           
    return ""
    

# for l in livres:
#     cont = []
#     typeAuteur = set(['(De (auteur))','(De (photographe))','(Coordination éditoriale de)'])
#     for a in l.contributeurs.split(','):
#         cont.append(extraireContributeur(a.strip()))    
#     print(cont)


## Création du graphe

In [59]:
from rdflib import Graph, Namespace, Literal, URIRef, BNode
from rdflib import RDF, RDFS, OWL, XSD


SCHEMA = Namespace("http://schema.org/")
HURTUBISE = Namespace("https://distributionhmh.com/onto/")
THEMA = Namespace("https://www.editeur.org/Thema/")


mapppingContributeurs = {
    '(De (auteur))': SCHEMA.author, 
    '(Coordination éditoriale de)': SCHEMA.editor, 
    '(Photographies de)': HURTUBISE.photographiesDe,
    '(Postface de)': HURTUBISE.postfaceDe,
    '(Préface de)': HURTUBISE.PrefaceDe,
    '(De (photographe))': SCHEMA.author,
    '(Traduit par)': SCHEMA.translator,
    '(Avant-propos de)': HURTUBISE.avantProposDe,
    '(Illustré par)': SCHEMA.illustrator,
    '(Texte de)': HURTUBISE.texteDe,
    '(Adapté par)': HURTUBISE.adaptePar
}



On va d'abord extraire les auteurs et autres contributeurs.
À noter que dans les données, il y a un champ pour les auteurs, mais on retrouve la même information dans la liste des contributeurs. Il y a par contre un différence. Dans le champs auteur, le nom de famille apparaît en premier, alors que dans la liste de contributeurs, il apparaît en dernier. 
On va donc prendre la liste de contributeurs.

In [ ]:

contributeurs = set()
for l in livres:
    for c in l.contributeurs.split(','):
        contributeurs.add(extraireContributeur(c.strip()))
        

# for c in contributeurs:
#     print(c)
    

contributeurs.remove('Collectif')
    
print(len(contributeurs), 'contributeurs au total')


    

In [60]:
grapheHurtubise = Graph()
grapheHurtubise.bind("schema","https://schema.org/", override=True)
grapheHurtubise.bind("hurtubise","https://distributionhmh.com/onto/", override=True)
grapheHurtubise.bind("thema","https://www.editeur.org/Thema/", override=True)



In [55]:
id = 0

uri_contributeur = {}

for c in contributeurs:
    id += 1
    uri_contributeur[c] = HURTUBISE['contributeur' + str(id)]
    grapheHurtubise.add((uri_contributeur[c], SCHEMA.name, Literal(c)))
    
    
print(len(grapheHurtubise), 'triplets dans le graphe de contributeurs')

11062 triplets dans le graphe de contributeurs


On peut maintenant ajouter les livres au graphe. Dans un premier temps, on crée les URI des livres. On stocke ces URI dans un dictionnaire, en utilisant les isbn comme clés.  

In [ ]:
id = 0

uri_livre = {}

for l in livres:
    id += 1
    grapheHurtubise.add((HURTUBISE['livre' + str(id)], RDF.type, SCHEMA.Book))
    uri_livre[l.isbn.papier + l.isbn.pdf +l.isbn.epub] = HURTUBISE['livre' + str(id)]
        

print(len(grapheHurtubise), 'triplets dans le graphe de contributeurs')        

Tel que spécifié précédemment, on va récupérer les auteurs dans la liste des contributeurs, plutôt que le champ "auteur". 
À noter ici qu'on suppose qu'il n'y a pas d'ambiguïté dans les noms d'auteurs.
À noter aussi qu'on a un cas spécial qui se présente parfois: les ouvrages collectifs, où une liste d'auteurs n'est pas toujours spécifiée.


In [ ]:
# Pour voir les cas où on a un ouvrage collectif
for l in livres:
    if "Collectif" in l.auteur:
        print(l.auteur)
        print(l.contributeurs)

In [ ]:
for l in livres:
    uri = uri_livre[l.isbn.papier + l.isbn.pdf +l.isbn.epub]
    
    for contributeur in l.contributeurs.split(','):
        contributeur = contributeur.strip()
        nom = extraireContributeur(contributeur)
        type = extraireTypeContributeur(contributeur)
        if nom == 'Collectif':
            bnode = BNode()
            grapheHurtubise.add((bnode, RDF.type, HURTUBISE.Collectif))
            grapheHurtubise.add((uri, mapppingContributeurs[type], bnode))

        else:
            grapheHurtubise.add((uri, mapppingContributeurs[type], uri_contributeur[nom]))

print(len(grapheHurtubise), 'triplets dans le graphe de contributeurs')                   

On va maintenant extraires les autres informations sur  l'oeuvre

In [ ]:
mappingLangue = {
    "Allemand": "de",
    "Anglais": "en",
    "Arabe": "ar",
    "Brésilien": "pt-BR",
    "Espagnol": "es",
    "Français": "fr",
    "Italien": "it",
    "Latin": "la",
    "Portugais": "pt",
}


In [82]:


for l in livres:
    uri = uri_livre[l.isbn.papier + l.isbn.pdf +l.isbn.epub]
    grapheHurtubise.add((uri, SCHEMA.name, Literal(l.titre)))
    
    if l.sous_titre:
        grapheHurtubise.add((uri, SCHEMA.alternativeHeadline, Literal(l.sous_titre)))
 
    if l.titre_serie:
        grapheHurtubise.add((uri, HURTUBISE.serie, Literal(l.titre_serie)))
        
    if l.langue:
        listeLangues = l.langue.split(', ')
        for langue in listeLangues:
            grapheHurtubise.add((uri, SCHEMA.inLanguage, Literal(mappingLangue[langue])))
        
    if l.langue_origine:   
        listeLangues = l.langue_origine.split(', ')
        for langue in listeLangues:
            grapheHurtubise.add((uri, HURTUBISE.langueOriginale, Literal(mappingLangue[langue])))
            
    if l.resume:
        grapheHurtubise.add((uri, SCHEMA.abstract, Literal(l.resume)))
        
    if l.annee_parution:
        grapheHurtubise.add((uri, SCHEMA.datePublished, Literal(l.annee_parution)))
        
    if l.thema_principal:
        thema = l.thema_principal.split(' - ')[0]
        grapheHurtubise.add((uri, HURTUBISE.thema_principal, THEMA[thema]))

    if l.thema_autre:
        thema = l.thema_autre.split(' - ')[0]
        grapheHurtubise.add((uri, HURTUBISE.thema_autre, THEMA[thema]))

        
    if l.quant_geo:
        thema = l.quant_geo.split(' - ')[0]
        grapheHurtubise.add((uri, HURTUBISE.thema_quant_geo, THEMA[thema]))

        
    if l.quant_langue:
        thema = l.quant_langue.split(' - ')[0]
        grapheHurtubise.add((uri, HURTUBISE.thema_quant_lang, THEMA[thema]))

        
    if l.quant_hist:
        thema = l.quant_hist.split(' - ')[0]
        grapheHurtubise.add((uri, HURTUBISE.thema_quant_hist, THEMA[thema]))

    if l.quant_interet:
        thema = l.quant_interet.split(' - ')[0]
        grapheHurtubise.add((uri, HURTUBISE.thema_quant_interet, THEMA[thema]))

    if l.quant_style:
        thema = l.quant_style.split(' - ')[0]
        grapheHurtubise.add((uri, HURTUBISE.thema_quant_style, THEMA[thema]))

    if l.quant_age:
        thema = l.quant_age.split(' - ')[0]
        grapheHurtubise.add((uri, HURTUBISE.thema_quant_age, THEMA[thema]))
 
    if l.mots_cles:
        for mot in l.mots_cles:
            grapheHurtubise.add((uri, HURTUBISE.mot_cle, Literal(mot)))
           

    if l.niveau_scolaire:
        for n in l.niveau_scolaire.split(', '):
            grapheHurtubise.add((uri, HURTUBISE.niveau_scolaire, Literal(n))) 
    if l.niveau_lecture:
        pass # Aucun item n'a cette information
    if l.echelle_CECR:
        pass # Aucun item n'a cette information
    if l.nombre_pages:
        grapheHurtubise.add((uri, SCHEMA.numberOfPages, Literal(l.nombre_pages, datatype=XSD.integer))) 
        
    if l.classification_editoriale:
        grapheHurtubise.add((uri, HURTUBISE.classification_editoriale, Literal(l.classification_editoriale))) 
        
        
print(len(grapheHurtubise), 'triplets dans le graphe de contributeurs')                   

10319 triplets dans le graphe de contributeurs


In [83]:
fichier_hurtubise = open("../Graphes/grapheHurtubise.rdf","wb")
grapheHurtubise.serialize(fichier_hurtubise) 
fichier_hurtubise.close()
